### Описание гипотезы
Проверим два предположения:
- Зависит ли вероятность оплаты от выбранного пользователем уровня сложности бесплатных тренировок?
- Существует ли разница во времени между пользователями с разным уровнем сложности и их первой оплатой?

Проверку будем производить на основе данных пользователей, которые зарегистрировались в 2017 году.

### Ход проверки

Импортируем нужные библиотеки

In [2]:
import pandas as pd
import psycopg2
import psycopg2.extras 
import numpy as np

Получаем данные из базы данных

In [ ]:
def getEventsData():
    query = '''SELECT e.* FROM case8.events e
    WHERE e.user_id in
    (SELECT DISTINCT(user_id) 
    FROM case8.events 
    WHERE event_type = 'registration' 
    AND start_time >= '2017-01-01'
    AND start_time < '2018-01-01')
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='lms.skillfactory.ru' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data

def getPurchaseData():
    query = '''SELECT p.* FROM case8.purchase p
    WHERE p.user_id in
    (SELECT DISTINCT(user_id) 
    FROM case8.events 
    WHERE event_type = 'registration' 
    AND start_time >= '2017-01-01'
    AND start_time < '2018-01-01')
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='lms.skillfactory.ru' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data

events_df = pd.DataFrame(getEventsData())
purchase_df = pd.DataFrame(getPurchaseData())

In [ ]:
events_df['start_time'] = pd.to_datetime(events_df['start_time'])
purchase_df['start_time'] = pd.to_datetime(purchase_df['start_time'])

In [ ]:
events_df = events_df.rename(columns={'id':'event_id'})
purchase_df = purchase_df.rename(columns={'id':'purchase_id'})

In [3]:

event = pd.read_csv('data/7_4_Events.csv') # чтение файла
mask = (event['start_time']>='2018-01-01') & (event['start_time']<'2019-01-01') & (event['event_type']=='registration') # условие фильтрации
registered = event[mask]['user_id'].to_list() # список всех зарегестрированных пользователей
events_df = event[event['user_id'].isin(registered)] # фильтрация таблицы по списку ID
events_df['start_time'] = pd.to_datetime(events_df['start_time']) # преобразование даты
events_df = events_df.rename(columns={"id": "event_id"}) # переименование колонок
events_df.head() # просмотр первых 5 строк таблицы

<ipython-input-3-8f8eea45653c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_df['start_time'] = pd.to_datetime(events_df['start_time']) # преобразование даты


,event_id,event_type,selected_level,start_time,tutorial_id,user_id
51405,80308,registration,NaN,2018-01-01 03:48:40,NaN,27832
51406,80309,registration,NaN,2018-01-01 04:07:25,NaN,27833
51407,80310,registration,NaN,2018-01-01 08:35:10,NaN,27834
51408,80311,registration,NaN,2018-01-01 11:54:47,NaN,27835
51409,80312,registration,NaN,2018-01-01 13:28:07,NaN,27836


In [4]:
""" Считываем данные таблицы Events, фильтруем только 2018 г и преобразовываем"""

purchase = pd.read_csv('data/purchase.csv') # чтение файла
purchase_df = purchase[purchase['user_id'].isin(registered)] # фильтрация таблицы по списку ID
purchase_df['event_datetime'] = pd.to_datetime(purchase_df['event_datetime']) # преобразование даты
purchase_df = purchase_df.rename(columns={"id": "purchase_id"}) # переименование колонок
purchase_df.head() # просмотр первых 5 строк таблицы

<ipython-input-4-f2722591ca90>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  purchase_df['event_datetime'] = pd.to_datetime(purchase_df['event_datetime']) # преобразование даты


,purchase_id,user_id,event_datetime,amount
1171,16845,27845,2018-01-03 18:53:43,100
1172,16846,27865,2018-01-04 14:46:10,250
1174,16848,27911,2018-01-07 08:19:12,50
1175,16849,27910,2018-01-07 12:11:34,100
1176,16850,27940,2018-01-07 13:16:41,200


Произведем преобразование типов (мы уже знаем из кейса, что его нужно проивзести, поэтому можно не делать обзор типов)

Перед объединением датафреймов, переименуем колонки

Сделаем объединенный датафрейм из событий и оплат

In [24]:
total_events_df = pd.concat([events_df,purchase_df],sort=False)
total_events_df['start_time'] = pd.to_datetime(total_events_df['start_time']) # преобразование даты
total_events_df['event_datetime'] = pd.to_datetime(total_events_df['event_datetime']) # преобразование даты

In [25]:
total_events_df

,event_id,event_type,selected_level,start_time,tutorial_id,user_id,purchase_id,event_datetime,amount
51405,80308.0,registration,NaN,2018-01-01 03:48:40,NaN,27832,NaN,NaT,NaN
51406,80309.0,registration,NaN,2018-01-01 04:07:25,NaN,27833,NaN,NaT,NaN
51407,80310.0,registration,NaN,2018-01-01 08:35:10,NaN,27834,NaN,NaT,NaN
51408,80311.0,registration,NaN,2018-01-01 11:54:47,NaN,27835,NaN,NaT,NaN
51409,80312.0,registration,NaN,2018-01-01 13:28:07,NaN,27836,NaN,NaT,NaN
...,...,...,...,...,...,...,...,...,...
2767,NaN,NaN,NaN,NaT,NaN,47498,18441.0,2019-01-02 03:48:19,100.0
2768,NaN,NaN,NaN,NaT,NaN,47647,18442.0,2019-01-02 23:26:26,150.0
2769,NaN,NaN,NaN,NaT,NaN,47554,18443.0,2019-01-03 00:36:36,50.0
2774,NaN,NaN,NaN,NaT,NaN,47742,18448.0,2019-01-04 12:51:41,50.0


Сделаем группы пользователей по уровню сложности, который был выбран для тренировок.

Сначала посмотрим как уровни сложности бывают:

In [26]:
total_events_df['selected_level'].unique()

array([nan, 'medium', 'hard', 'easy'], dtype=object)

Сформируем группы

In [27]:
users_with_easy_level = total_events_df[total_events_df['selected_level'] == 'easy']['user_id'].unique()
users_with_medium_level = total_events_df[total_events_df['selected_level'] == 'medium']['user_id'].unique()
users_with_hard_level = total_events_df[total_events_df['selected_level'] == 'hard']['user_id'].unique()

Посчитаем для каждой группы показатели

In [28]:
user_groups = [
    {'easy': users_with_easy_level},
    {'medium': users_with_medium_level},
    {'hard': users_with_hard_level}
]

for group in user_groups:
    level = list(group.keys())[0]
    group_users = group[level]
    count_of_users_in_group = len(group_users)
    purchase_df_slice = purchase_df[purchase_df['user_id'].isin(group_users)]
    percent_of_purchase = purchase_df_slice['user_id'].nunique()/count_of_users_in_group
    print ('Процент оплативших пользователей, выбравших уровень сложности {}: {:.2%}'.format(level,percent_of_purchase))
    print ()
    level_choice_df = total_events_df[(total_events_df['event_type'] == 'level_choice') & (total_events_df['user_id'].isin(group_users))]
    if (level_choice_df['user_id'].value_counts().mean()) == 1:
        level_choice_df = level_choice_df[['user_id','start_time']].rename(columns={'start_time':'level_choice_time'})
        purchase_df_slice_2 = purchase_df_slice[['user_id','start_time']].rename(columns={'start_time':'purchase_time'})
        merged_df = purchase_df_slice_2.merge(level_choice_df,on='user_id',how='inner')
        merged_df['timedelta'] = merged_df['purchase_time'] - merged_df['level_choice_time']
        mean_time = merged_df['timedelta'].mean()
        print ('Среднее время между выбором уровня сложности и оплатой для пользователей, выбравших уровень сложности {}: {}'.format(level,mean_time))
        print ('Характеристики времени:')
        print (merged_df['timedelta'].describe())
    else:
        print ('Более 1 события выбора уровня сложности')

Процент оплативших пользователей, выбравших уровень сложности easy: 7.72%



KeyError: "['start_time'] not in index"

### Выводы
Процент оплативших пользователей, выбравших уровень сложности easy: 7.72%
Процент оплативших пользователей, выбравших уровень сложности medium: 20.86%
Процент оплативших пользователей, выбравших уровень сложности hard: 35.39%

Таким образом, существует прямая зависимость между выбранным уровнем сложности и процент пользователей, которые оплатили тренировки. Пользователи, выбирающие более сложные тренировки, чаще покупают платные тренировки.

Наибольшее время между событием выбора уровня сложности и моментом первой оплаты, тратят пользователи, которые выбрали уровень сложности medium. Для них среднее время составляет 3 дня 23 часа, а медианное время 3 дня 19 часов.

Чуть меньше времени тратят пользователи, выбравшие уровень сложности easy. Для них среднее время составляет 3 дня 14 часов, а медианное время 3 дня 6 часов.

Меньше всего времени на принятие решение о покупке тратят пользователя, выбравшие уровень сложности hard. Для них среднее время составляет 3 дня 7 часов, а медианное время 3 дня 13 часов. 

**Пожалуйста, заполните небольшую анкету обратной связи по эталонному ответу для кейса #8:**

https://forms.gle/z2VAj6R49yYuxqGGA